# Project
 - 2 CLUSTERING AND 1 DIMENSION REDUCTION METHOD NOT LEARNT IN THE COURSE 

## Static Dataset

In [1]:
import numpy as np
from sklearn.mixture import GaussianMixture
from tqdm import tqdm
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, Birch
from sklearn.manifold import TSNE, SpectralEmbedding, Isomap, MDS
from fcmeans import FCM
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

import random
import networkx as nx
from community import community_louvain

### Cluster the remaining columns, and explain what are is the best clustering method.
- silhouette_score
- elbow method
- k-fold cross validation on silhouette_score or Matual information with - kruskal wallis or anova

In [ ]:
def get_best_model_by_silhouette(models, X)
    for model in models:
        labels = model.fit_predict(X)
        scores.append(metrics.silhouette_score(X, labels))
    plt.plot(models, scores)
    plt.title(f"silhouette_score diffrent models - {model_name}")
    plt.xlabel("models")
    plt.ylabel("Silhouette Score")

### Fuzzy C-Means

In [ ]:
my_model = FCM(n_clusters=2)
my_model.fit(X)

#### K-Means

In [ ]:
KMeans(k, n_init="auto", random_state=random_state)

#### Gaussian-Mixture

In [ ]:
GaussianMixture(k, max_iter=3000)

#### Hierarchical Clustering

In [ ]:
import scipy.cluster.hierarchy as sch # importing scipy.cluster.hierarchy for dendrogram
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward')) # finding the optimal number of clusters using dendrogram
plt.title('Dendrogram') # title of the dendrogram
plt.xlabel('Customers') # label of the x-axis
plt.ylabel('Euclidean distances') # label of the y-axis
plt.show() # show the dendrogram

In [ ]:
Agg_hc = AgglomerativeClustering(n_clusters = 5, affinity = 'euclidean', linkage = 'ward')
y_hc = Agg_hc.fit_predict(newData)

#### Birch - didn't learned in the class

In [ ]:
Birch(n_clusters=k)

#### Dbscan

In [ ]:

def get_best_dbscan_params(clean_X: np.ndarray, min_samples: int):
    range_eps = np.linspace(0.1, 10, 20)
    scores = []
    for eps in range_eps:
        model = DBSCAN(eps=eps, min_samples=min_samples)
        good_labels = model.fit_predict(clean_X)
        noisy_data_count = len(good_labels[good_labels == -1])
        if noisy_data_count > good_labels.shape[0] * 0.5 or len(np.unique(good_labels)) < 3:
            scores.append(0)
            continue
        print(f"noisy data count with eps={eps}: {noisy_data_count}")
        try:
            score = metrics.silhouette_score(clean_X, good_labels)
        except:
            score = 0
        scores.append(score)
    plt.plot(range_eps, scores)
    plt.show()
    return range_eps[np.argmax(scores)]


#### SpectralClustering

In [ ]:
clustering = SpectralClustering(n_clusters=2,assign_labels='discretize',random_state=0).fit_predict(X)

### Associate the clusters with the ground truth, and explain what external variable is best associated with clusters, and what clustering method best associates with the outside variables

### Find anomalies in the data

### using DBSCAN

In [ ]:
model = DBSCAN(eps=eps, min_samples=min_samples)
labels = model.fit_predict(X)
labels[labels == -1]

### using isolation forest
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html

In [ ]:
clf = IsolationForest(random_state=0).fit(X)

### one classs svm

In [ ]:
one_class_svm = OneClassSVM(nu=0.01, kernel = 'rbf', gamma = 'auto').fit(X_train)
prediction = one_class_svm.predict(X_test)

### test whether anomalies are associated with any of the external variables

### check if anomaly detection improves the clustering

### Reduce the dimension of the data and propose a visualization that best characterize the clusters associated with the external variables. Please find a smart presentation scheme to highlight both clusters and variables.

- TSNE
- Isomap
- MDS
- PCA
- SpectralEmbedding
- PCoA
- LLE
- UMAP
- AutoEncoders
- ICA

In [8]:
def reduce_dimension(X):
    models = [TSNE(n_components=2), Isomap(n_components=2), MDS(n_components=2), SpectralEmbedding(n_components=2)]
    fig, axs = plt.subplots(nrows=len(models), ncols=1, figsize=(8, 8))
    fig.tight_layout()
    for i, model in tqdm(enumerate(models)):
        X_embedded_tsne = model.fit_transform(X)
        axs[i].scatter(X_embedded_tsne[:, 0], X_embedded_tsne[:, 1], s=40, cmap='viridis')
        axs[i].set_title(f"{model} dimensionality reduction")
    plt.show()

## Dynamic Dataset

### Hidden Markov model for time series
https://rubikscode.net/2021/09/06/stock-price-prediction-using-hidden-markov-model/

## Graph Based Dataset

### Louvain’s Algorithm for Community Detection
https://github.com/taynaud/python-louvain

In [9]:

def generate_network(n):
    # initialize dictionary with nodes
    graph_dct = {node:[] for node in range(n)}
    nodes = list(range(n))
    
    # generate edges
    for n,edge_list in graph_dct.items():
        edge_c = random.randint(min(nodes), int(max(nodes) / 2))
        el = random.sample(nodes, edge_c)
        graph_dct[n] = el
    
    # create networkx multi-edge graph
    G = nx.MultiGraph(graph_dct)
    return G

n = 500
G = generate_network(n)
print(nx.info(G))

# visualize graph
pos = nx.spring_layout(G)
nx.draw(G, pos, node_size = 75, alpha = 0.8)
plt.show()

In [ ]:
comms = community_louvain.best_partition(G)

In [ ]:
unique_coms = np.unique(list(comms.values()))
cmap = {
    0 : 'maroon',
    1 : 'teal',
    2 : 'black', 
    3 : 'orange',
    4 : 'green',
    5 : 'yellow'
}

node_cmap = [cmap[v] for _,v in comms.items()]

pos = nx.spring_layout(G)
nx.draw(G, pos, node_size = 75, alpha = 0.8, node_color=node_cmap)
plt.show()